# My First Agent


I am following the tutorial from this webpage:
https://python.langchain.com/docs/tutorials/

My goal is to create something that anyone can run from COLAB.

### Install Relevant Libraries

In [77]:
pip install langchain

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [78]:
pip install -qU "langchain[openai]"

Note: you may need to restart the kernel to use updated packages.


In [79]:
pip install langchain-community pypdf

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [126]:
pip install langchain-core langgraph>0.2.27

Note: you may need to restart the kernel to use updated packages.


In [80]:
pip install --upgrade --quiet langchain-core

Note: you may need to restart the kernel to use updated packages.


In [81]:
pip install -qU langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [82]:
pip install --upgrade langchain-core


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### SetUp Logging

In [83]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

### Connect to OPEN AI

In [84]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

## Prompting




In [85]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Be58KyQPdATSuSybSAV8RGibXtsE8', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--bbb628d9-3a2e-474d-8d97-69d210aaa989-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Quick Note, the following are equivalent:  

model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")]) 


### To see how many tokens are in the answer:  

In [86]:
for token in model.stream(messages):
    print(token.content, end="|")

|C|iao|!||

### Prompt Template


For more prompt templates, go here:  

https://python.langchain.com/docs/how_to/#prompt-templates

In [87]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [88]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [89]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

## Invoking the llm

This uses the prompts we set up and runs the LLM

In [90]:
response = model.invoke(prompt)
print(response.content)

Ciao!


## Documents and Document Loading

#### An Example of very simple documents

In [91]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

#### Loading PDF's 

Be sure to update the path here

In [92]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


This splits it into 1 page ==  1 doc. We may need something a bit more granular

In [93]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


#### Using Langchain to split the text

In [94]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

516

## Embeddings 

I need an embeddings model too.

In [95]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [96]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 3072

[0.009356783702969551, -0.01619010977447033, 0.00035251901135779917, 0.006321138236671686, 0.020612407475709915, -0.03930098935961723, -0.007439204957336187, 0.04112487658858299, -0.008088808506727219, 0.0595136396586895]


## Vector Stores 


in Langchain Vectore objects contain methods for adding text and Document Objects to the store, and quering them using various similiarity metics.  

They are oftem initialized with embedding models, which determine how text data is translated into numeric vectors.

#### Using chromaDB

In [97]:
from langchain_chroma import Chroma

# Instantiating the vector store
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [98]:
# Indexing the documents in the database

ids = vector_store.add_documents(documents=all_splits)

### Return Documents based on similiarity to a string query

In [99]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,' metadata={'subject': 'Form 10-K filed on 2023-07-20 for the p

In [100]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"
"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, equipment, accessories and services. NIKE is
the largest seller of athletic footwear and apparel in the world. We sell our products through NIKE Direct operations, which are comprised of both NIKE-owned retail stores
and sales through our digital platforms (also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales' metadata={'start_index': 0, 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'total_pages': 107, 'page':

### Return Similiarity scores

In [101]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.6225528717041016

page_content='Table of Contents
FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTSThe following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and major product line:
FISCAL 2023 COMPARED TO FISCAL 2022
• NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.
The increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6,
2 and 1 percentage points to NIKE, Inc. Revenues, respectively.
• NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This
increase was primarily due to higher revenues in Men's, the Jordan Brand, Women's and Kids' which grew 17%, 35%,11% and 10%, respectively, on a wholesale
equivalent basis.' metad

In [102]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Table of Contents
GROSS MARGIN
FISCAL 2023 COMPARED TO FISCAL 2022
For fiscal 2023, our consolidated gross profit increased 4% to $22,292 million compared to $21,479 million for fiscal 2022. Gross margin decreased 250 basis points to
43.5% for fiscal 2023 compared to 46.0% for fiscal 2022 due to the following:
*Wholesale equivalent
The decrease in gross margin for fiscal 2023 was primarily due to:
• Higher NIKE Brand product costs, on a wholesale equivalent basis, primarily due to higher input costs and elevated inbound freight and logistics costs as well as
product mix;
• Lower margin in our NIKE Direct business, driven by higher promotional activity to liquidate inventory in the current period compared to lower promotional activity in
the prior period resulting from lower available inventory supply;
• Unfavorable changes in net foreign currency exchange rates, including hedges; and
• Lower off-price margin, on a wholesale equivalent basis.
This was partially offset by:'

## Retrievers

In [103]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='14b2fdd1-9313-477c-ae19-cfa69b0ef99d', metadata={'total_pages': 107, 'page_label': '27', 'start_index': 804, 'creationdate': '2023-07-20T16:22:00-04:00', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'title': '0000320187-23-000039', 'source': 'nke-10k-2023.pdf', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'page': 26, 'moddate': '2023-07-20T16:22:08-04:00', 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'keywords': '0000320187-23-000039; ; 10-K', 'creator': 'EDGAR Filing HTML Converter'}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which

In [104]:
# This converts a vector store into a retriever

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='14b2fdd1-9313-477c-ae19-cfa69b0ef99d', metadata={'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'creationdate': '2023-07-20T16:22:00-04:00', 'page_label': '27', 'moddate': '2023-07-20T16:22:08-04:00', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'source': 'nke-10k-2023.pdf', 'start_index': 804, 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'title': '0000320187-23-000039', 'page': 26, 'creator': 'EDGAR Filing HTML Converter', 'keywords': '0000320187-23-000039; ; 10-K', 'total_pages': 107}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which

## Classifying Text into Labels

In [105]:
pip install --upgrade --quiet langchain-core

Note: you may need to restart the kernel to use updated packages.


#### Login to Open AI

In [106]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

#### Specifying a pre-build model from Pydantic

In [107]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# Structured LLM
structured_llm = llm.with_structured_output(Classification)

In [108]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

#### Having a bit more fine tune over the model

In [109]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [110]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(
    Classification
)

In [111]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='happy', aggressiveness=1, language='spanish')

In [112]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='sad', aggressiveness=4, language='spanish')

## Extraction - Intro to Tool Calling

### Extraction Schema

In [113]:
# Setup logging in LangSmith

import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [114]:
from typing import Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

In [115]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

### Choose model

In [116]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [117]:
structured_llm = llm.with_structured_output(schema=Person)

In [118]:
text = "Alan Smith is 6 feet tall and has blond hair."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='Alan Smith', hair_color='blond', height_in_meters='1.83')

### Extracting Lists of Entities  --- rather than a single entity

In [119]:
from typing import List, Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )


class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

In [120]:
structured_llm = llm.with_structured_output(schema=Data)
text = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Data(people=[Person(name='Jeff', hair_color='black', height_in_meters='1.83'), Person(name='Anna', hair_color='black', height_in_meters=None)])

### Few Shot Prompting Example

In [121]:
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = llm.invoke(messages)
print(response.content)

7


### Using tools - tool texample to messages

In [122]:
from langchain_core.utils.function_calling import tool_example_to_messages

examples = [
    (
        "The ocean is vast and blue. It's more than 20,000 feet deep.",
        Data(people=[]),
    ),
    (
        "Fiona traveled far from France to Spain.",
        Data(people=[Person(name="Fiona", height_in_meters=None, hair_color=None)]),
    ),
]


messages = []

for txt, tool_call in examples:
    if tool_call.people:
        # This final message is optional for some providers
        ai_response = "Detected people."
    else:
        ai_response = "Detected no people."
    messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))

In [123]:
for message in messages:
    message.pretty_print()

================================ Human Message =================================

The ocean is vast and blue. It's more than 20,000 feet deep.
================================== Ai Message ==================================
Tool Calls:
  Data (c0d836e0-89a3-481c-afda-33d0585f2595)
 Call ID: c0d836e0-89a3-481c-afda-33d0585f2595
  Args:
    people: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

Detected no people.
================================ Human Message =================================

Fiona traveled far from France to Spain.
================================== Ai Message ==================================
Tool Calls:
  Data (529e93e4-33db-4f50-af6a-03e6b4fc0d4f)
 Call ID: 529e93e4-33db-4f50-af6a-03e6b4fc0d4f
  Args:
    people: [{'name': 'Fiona', 'hair_color': None, 'height_in_meters': None}]
================================

In [124]:
message_no_extraction = {
    "role": "user",
    "content": "The solar system is large, but earth has only 1 moon.",
}

structured_llm = llm.with_structured_output(schema=Data)
structured_llm.invoke([message_no_extraction])

Data(people=[])

In [125]:
structured_llm.invoke(messages + [message_no_extraction])

Data(people=[])

## Building a ChatBot

In [127]:
# Loggin with Langsmith

# Setup logging in LangSmith

import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [129]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [130]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Be5H3p6KaqecZdGmOiwE6rFF3vt7m', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3cc3aa1e-237f-4b74-ab5f-c96137e3f748-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Notice it is Memoryless

In [131]:
model.invoke([HumanMessage(content="What's my name?")])



AIMessage(content="I'm sorry, but I don't have access to personal data about users unless it has been shared with me in this conversation. I can't tell your name. If you'd like to share your name or ask about something else, feel free!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 11, 'total_tokens': 57, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Be5HxzHAb6PaCfQIzIxtpRKnj2jNp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c05024f0-bce5-46da-8d6e-0df36b9aa066-0', usage_metadata={'input_tokens': 11, 'output_tokens': 46, 'total_tokens': 57, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'r

### Manually Passing the entire conversation to the chatbot

In [143]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob! How can I help you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 33, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Be5P5lXRoYpcLfyD8l8l6tqIgsJPh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d2647652-66fa-41e1-b71d-375d7ecd3720-0', usage_metadata={'input_tokens': 33, 'output_tokens': 14, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Let's use the libraries to add memory

In [144]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [145]:
config = {"configurable": {"thread_id": "abc123"}}

''' This enables us to support multiple conversation threads with a single application, a common requirement when your application has multiple users'''

' This enables us to support multiple conversation threads with a single application, a common requirement when your application has multiple users'

In [146]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! How can I assist you today?


In [147]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! How can I help you today, Bob?


### Changing thread ID resets memory

In [148]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about you unless you've shared it in this conversation. How can I assist you today?


In [149]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. If you have any other questions or need assistance, feel free to ask!


#### FOr async support, update the call_model node

Update it to be an async function and use .ainvoke when invoking the application.

In [150]:
# # Async function for node:
# async def call_model(state: MessagesState):
#     response = await model.ainvoke(state["messages"])
#     return {"messages": response}


# # Define graph as before:
# workflow = StateGraph(state_schema=MessagesState)
# workflow.add_edge(START, "model")
# workflow.add_node("model", call_model)
# app = workflow.compile(checkpointer=MemorySaver())

# # Async invocation:
# output = await app.ainvoke({"messages": input_messages}, config)
# output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about you unless you've shared it in the conversation. How can I assist you today?


## Prompt Templates

In [151]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [152]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [153]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy, Jim! Be ye seekin’ treasure or just wishin' fer a bit o’ parley? What can this ol’ sea dog do fer ye today?


In [154]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yer name be Jim, savvy? A fine name fer a fine matey! What other inquiries can I help ye with, Jim?


### Making things slightly more complicated

In [155]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [156]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [157]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob! ¿Cómo puedo ayudarte hoy?


In [159]:
# Notice how state is persistent

query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

# We didn't have to set "language" : language inside app.invoke({ },config)

================================== Ai Message ==================================

Tu nombre es Bob.


## Managing Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.

In [175]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=85,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [176]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

Now, the conversation won't remember our name since we trimmed out that much history

In [177]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. Could you tell me?


But it remembers more recent stuff.

In [178]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked what 2 + 2 equals.


Stream the messages back to us.

In [184]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hi| Todd|!| Here|’s| another| joke| for| you|:

|Why| don|’t| skeleton|s| fight| each| other|?

|They| don|’t| have| the| guts|!||

# Build an Agent

### Setting up Websearch Tool

In [194]:
pip install -qU langchain-tavily

Note: you may need to restart the kernel to use updated packages.


In [195]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [189]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4", model_provider="openai")

In [190]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [196]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Thursday, March 6, 2025. San Francisco, CA - Weather Forecast', 'url': 'https://weathershogun.com/weather/usa/ca/san-francisco/480/march/2025-03-06', 'content': 'San Francisco, California Weather: Thursday, March 6, 2025. Partly sunny weather with scattered clouds and occasional rain showers.', 'score': 0.94536424}, {'title': 'Weather in San Francisco in March 2025 (California)', 'url': 'https://world-weather.info/forecast/usa/san_francisco/march-2025/', 'content': 'Weather in San Francisco in March 2025 San Francisco Weather Forecast for March 2025 is based on statistical data. March +50° +50° +48° +50° +52° +46° +46° +46° +46° +46° +48° +54° +46° +46° +48° +50° +52° +46° +45° +46° +48° +50° +50° +50° +59° +55° +55° +54° +48° +50° +54° Average weather in March 2025 Extended weather forecast in San Francisco Weather in large and nearby cities Weather in Washington, D.C.+55° Sacramento+82° Pleasanton+73° Redwood City+68° San Leandro+64° San Mateo+64° San Rafael+66° San Ramon

In [201]:
model_with_tools = model.bind_tools(tools)


In [198]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [203]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_RDdINMXfa3RUiJjspZBB8vTT', 'type': 'tool_call'}]


Note, this is not calling the tool yet, it is just telling us to call the tool

# Create the Agent

In [204]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [205]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='d924ae6a-2ded-4680-b47c-6e795a44abfc'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BeDZZ1mWxMnEivD4bM9ZEbcZN9Lcq', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--991e3cd0-e259-4944-bc82-a55bf505cd7e-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [206]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='8b028eaa-31a7-4701-b7ce-a9ad4e9d3937'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7TI0mZSf6y0Y9qFoRwzeio6X', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BeDeBokThtDkL2gDOhnlrA4B7n5sU', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c347e04d-8010-4a9a-94c8-9e76a0c1ff72-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query

### Streaming Style

In [209]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()


# Or to stream token level:

# for step, metadata in agent_executor.stream(
#     {"messages": [HumanMessage(content="whats the weather in sf?")]},
#     stream_mode="messages",
# ):
#     if metadata["langgraph_node"] == "agent" and (text := step.text()):
#         print(text, end="|")

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_2pZmZOSlhn44Dxa0vv88QMqL)
 Call ID: call_2pZmZOSlhn44Dxa0vv88QMqL
  Args:
    query: current weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Thursday, March 6, 2025. San Francisco, CA - Weather Forecast", "url": "https://weathershogun.com/weather/usa/ca/san-francisco/480/march/2025-03-06", "content": "San Francisco, California Weather: Thursday, March 6, 2025. Partly sunny weather with scattered clouds and occasional rain showers.", "score": 0.9245858}, {"title": "March 2025 Weather History in San Francisco California, United ...", "url": "https://weatherspark.com/h/m/557/2025/3/Historical-Weather-in-March-2025-in-San-Francisco-California-United-States"

## Adding Memory to the Agent

In [210]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [211]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [212]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello, Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 85, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BeDh93spQwIIgDcAcQ4a7RnRFLWfJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9055b927-b767-41d1-9b07-d1bfae28dc42-0', usage_metadata={'input_tokens': 85, 'output_tokens': 12, 'total_tokens': 97, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [214]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 127, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BeDhLB7KP8luAB42U6eeX2Jidu8by', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a33611eb-6abe-4dee-a432-7ff71458fa56-0', usage_metadata={'input_tokens': 127, 'output_tokens': 6, 'total_tokens': 133, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


## RAG Part 1


This part teaches how to access documents.

In [216]:
pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph


Note: you may need to restart the kernel to use updated packages.


In [217]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [218]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [219]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [220]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering

prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

## Rag Part II

In [ ]:
# Revisit the vector store from part 1

import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

## Turning Retrival step into a tool

In [ ]:
from langchain_core.tools import tool


@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

We build them below. Note that we leverage another pre-built LangGraph component, ToolNode, that executes the tool and adds the result as a ToolMessage to the state.



In [ ]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode


# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

### Compile application into a single graph object.

Finally, we compile our application into a single graph object. In this case, we are just connecting the steps into a sequence. We also allow the first query_or_respond step to "short-circuit" and respond directly to the user if it does not generate a tool call. This allows our application to support conversational experiences-- e.g., responding to generic greetings that may not require a retrieval step

In [ ]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [ ]:
input_message = "Hello"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = "What is Task Decomposition?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

### Adding Memory

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# Specify an ID for the thread
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
input_message = "What is Task Decomposition?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = "Can you look up some common ways of doing it?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

## Agents w/ RAG and Memory


In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, [retrieve], checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "def234"}}

input_message = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent_executor.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    event["messages"][-1].pretty_print()

## Summarizing Many Papers

#### Loading a Database

In [15]:
# Import Relevant Libraries
import getpass
import os
import glob
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader  # Added missing import
import time
from tqdm import tqdm
import warnings

# Suppress PDF parsing warnings
warnings.filterwarnings("ignore", category=UserWarning)
from langchain_community.document_loaders import WebBaseLoader
from langchain.chat_models import init_chat_model

# Keys for tracing and API
os.environ["LANGSMITH_TRACING"] = "true"
if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter LangSmith API Key: ")
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.2)

def load_pdfs_from_folder(folder):
    """Load PDFs and return each as a separate document without combining pages"""
    pdf_paths = glob.glob(f"{folder}/*.pdf")
    docs = []
    for i, path in enumerate(pdf_paths):
        print(f"Loading PDF {i+1}/{len(pdf_paths)}: {os.path.basename(path)}")
        try:
            loader = PyPDFLoader(path)
            pdf_docs = loader.load()
            
            # Keep each page separate to avoid context window issues
            # But combine into logical chunks if needed
            full_text = ""
            for page in pdf_docs:
                full_text += page.page_content + "\n"
            
            # Only add if we successfully extracted text
            if full_text.strip():
                docs.append({
                    "path": path, 
                    "content": full_text,
                    "filename": os.path.basename(path)
                })
            else:
                print(f"Warning: No text extracted from {path}")
                
        except Exception as e:
            print(f"Error loading {path}: {e}")
            continue
    
    return docs

# Field description
# Field description
MY_FIELD_DESCRIPTION = """My field is Multirobot Adaptive Navigation in Environmental Vector Fields, 
combining robotics, differential geometry, and information theory to determine the absolute minimum 
information required for navigation in complex environments. The core discovery is that environmental 
vector fields contain sufficient geometric structure for complete navigation using only instantaneous 
measurements from 3 or 4 strategically positioned robots, which can extract all necessary second-order field 
information (gradients, Jacobians, Hessians, eigenstructure) that traditionally required 6 or more 
measurements. The key breakthrough is developing memory-free control laws that achieve convergence using 
only instantaneous 10Hz measurements without any state estimation or feedback history, exploiting 
fundamental mathematical properties like Hessian symmetry and vector field consistency constraints where 
the field structure itself acts as a natural computer providing navigation instructions through classical 
optimization geometry. Technical innovations include proving that 4-point multi-robot formations form sufficient 
sampling stencils for second-order field properties, creating universal navigation primitives that handle 
saddle points in sclar fieldss, three robot primitives that can attract, repel, and maintain a fixed orbit around
critical points in 2D vector fields (focus, nodes, centers, vortices, saddle points) through a single memoryless orbit primitive, and 
achieving sub-centimeter formation precision enabling reliable field geometry extraction.
Another contribution is a primitive that moves through a vector field along the separatrix or bifurcation. This approach 
enables applications in GPS-denied environments like ocean robots tracking pollution plumes with minimal 
battery power, aerial swarms characterizing atmospheric phenomena with limited communication, and 
search-and-rescue teams following chemical gradients, with all theoretical results validated on a 
physical testbed marking the first successful implementation of truly information-minimal navigation 
bridging the simulation-to-reality gap."""

# Analysis questions
ANALYSIS_QUESTIONS = """
Analyze this paper and extract the following information:

1. Main Contributions: What are the 2-4 key contributions claimed by the authors?
2. Objective: What was their objective, and how well did they achive it (Give metrics)
3. Limitations: What limitations did the identify to their work (Identify 2-5)
4. Future Work: What future directions do they see their work going in (Identify 2 -5)
5. Methods: What hardware and software were used? Was it open source or custom.
6. Field Positiong: How do the authors describe the current state of their own field? Be very verbose here (like 100 words)
7. Multirobot fields: How do the authors describe the current state of the art in multirobot or multiagent systems? Very verbose again.
8. Research Gaps: Does the author identify any outstanding challenges in their field, or areas that are not explored well.
9. Related Work Section: What other papers do they cite as most relevant? (note ~10 key references) Copy these citations exactly.
10. Jacobians and Hessians: Does it estimate a Jacobian or Hessian from Distributed measurements? If so, how?
10. Physical Implementation: Is it all in sim, or do they have some real world testing also? If so, how?
11. Vector Fields: Does this work pertain to environmental vector fields, or to Artificial vector fields?
12. Novelty and Elegance: Do any of these papers use similar techniques as my approach, solve the same problem I'm addressing in a different method, makes a claim that would encompass my work?
13. Adaptive and Reactive: Is the approach reactive using only current readings, or does it need multiple readings (like slam?).
14. Memory requirements: Does this paper require memory or state history for navigation decisions, and if so, what specific information must be stored between time steps? If control is involved, does it mention discrete steps or continous control laws?
15. What branches of mathematics are being used? 
16. Formation Control: What formation is required or achieved, and how does formation accuracy affect robots?
17. Scalability: Does the paper address scalability? Does it address the minimum amount of information needed to achieve success?
18. Practicality and Usability: Are there any practical examples for the research?
19. Fail Methods: Does the method degrade gracefully when formation is lost or robots drift from prescribed positions? What about if robots fail in communication?
20. Key words: Does it mention any of these key words, if so, in what context? Focii, saddle, centre, vortex, sink, source, separatrix, bifurcation, memoryless navigation, eigenvector, eigenvalue, reactive navigation, adapative navigation, eigenstructure, determinant.
21. Relevance: on a scale from 1-10, how relevant is this to my research? Explain your reasoning.
22. Similiarities and differences. Explain how my research addresses any gaps or future work identified in the paper. (or n/a and explanation.)


Please be specific and quote directly when the authors make important claims about the field or gaps. Be verbose, as I want to extract as much context as possible out of each paper as it related to my field as possible.
"""

# Create the review prompt
review_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a highly analytical academic reviewer.\nUser's field: " + MY_FIELD_DESCRIPTION),
    ("human", "Given the following paper text, answer these questions:\n"
     + ANALYSIS_QUESTIONS +
    "\n=== BEGIN PAPER DOCUMENT ===\n{paper_text}\n=== END PAPER DOCUMENT ===")
])

def analyze_papers(folder_path="./PapersForResearch", output_file="all_paper_summaries.md"):
    """Main function to analyze all papers and generate summaries"""
    
    # Load PDFs
    print("Loading PDFs...")
    pdf_docs = load_pdfs_from_folder(folder_path)
    print(f"Successfully loaded {len(pdf_docs)} papers")
    
    if not pdf_docs:
        print("No PDFs found or loaded successfully!")
        return
    
    # Process each paper
    results = []
    
    for i, doc in enumerate(pdf_docs):
        print(f"\n=== Processing paper {i+1}/{len(pdf_docs)}: {doc['filename']} ===")
        
        # Check if content is too long (rough estimate for token limit)
        if len(doc['content']) > 100000:  # Adjust this threshold as needed
            print(f"Warning: {doc['filename']} is very long ({len(doc['content'])} chars). Consider splitting.")
        
        # Prepare input
        paper_input = {"paper_text": doc['content']}
        
        # Create chain
        chain = review_prompt | llm
        
        # Generate analysis
        try:
            print(f"Sending to LLM...")
            response = chain.invoke(paper_input)
            summary = response.content if hasattr(response, 'content') else str(response)
            print(f"✓ Successfully analyzed {doc['filename']}")
            
        except Exception as e:
            summary = f"ERROR analyzing {doc['filename']}: {e}"
            print(f"✗ Error analyzing {doc['filename']}: {e}")
        
        # Store result
        results.append({
            "file": doc["path"],
            "filename": doc["filename"],
            "summary": summary,
        })
        
        # Rate limiting
        time.sleep(2)  # Adjust based on your OpenAI plan
    
    # Write all results to file (FIXED: moved outside the loop)
    print(f"\nWriting results to {output_file}...")
    with open(output_file, "w", encoding='utf-8') as f:
        f.write(f"# Academic Paper Analysis Results\n\n")
        f.write(f"Generated on: {time.strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write(f"Total papers analyzed: {len(results)}\n\n")
        f.write("---\n\n")
        
        for r in results:
            f.write(f"# Analysis for {r['filename']}\n\n")
            f.write(f"**File Path:** {r['file']}\n\n")
            f.write(r['summary'])
            f.write("\n\n" + "="*80 + "\n\n")
    
    print(f"✓ Analysis complete! Results saved to {output_file}")
    return results

# Run the analysis
if __name__ == "__main__":
    # You can customize these parameters
    results = analyze_papers(
        folder_path="./PapersForResearch",  # Change this to your PDF folder
        output_file="all_paper_summaries.md"
    )

Loading PDFs...
Loading PDF 1/60: [2] INDOOR TESTBED FOR VECTOR FIELD MULTIROBOT ADAPTIVE NAVIGATION (2).pdf
Loading PDF 2/60: [33] On_the_Guidance__Navigation_and_Control_of_In_orbit_Space_Robotic_Missions__A_Survey_and_Prospective_Vision.pdf
Loading PDF 3/60: [27] Augmented_Kalman_Filter_Design_in_a_Localization_System_Using_Onboard_Sensors_With_Intrinsic_Delays.pdf
Loading PDF 4/60: [23] Safe multiagent reinforcement learning.pdf
Loading PDF 5/60: [57] Matroid s10514-018-9778-6.pdf
Loading PDF 6/60: [13] Multi-Robot_Dynamical_Source_Seeking_in_Unknown_Environments.pdf
Loading PDF 7/60: [44] Fully_Decentralized_Controller_for_Multi-Robot_Collective_Transport_in_Space_Applications.pdf
Loading PDF 8/60: [58] Differential_analysis_of_bifurcations_and_isolated_singularities_for_robots_and_mechanisms.pdf
Loading PDF 9/60: [22] Mobile_Robot_Navigation_Functions_Tuned_by_Sensor_Readings_in_Partially_Known_Environments.pdf
Loading PDF 10/60: [40] An_Autonomous_Navigation_Strategy_Based_on_Im

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 57 0 (offset 0)
Ignoring wrong pointing object 77 0 (offset 0)
Ignoring wrong pointing object 79 0 (offset 0)
Ignoring wrong 

Loading PDF 33/60: [30] Final Paper.pdf
Loading PDF 34/60: [7] Guiding_Vector_Fields_for_the_Distributed_Motion_Coordination_of_Mobile_Robots.pdf
Loading PDF 35/60: [14] A Survey of Distributed Relative Localization Algorithms.pdf
Loading PDF 36/60: [51] Optimizing_Topologies_for_Probabilistically_Secure_Multi-Robot_Systems.pdf
Loading PDF 37/60: [3] Initial_Study_of_Multirobot_Adaptive_Navigation_for_Exploring_Environmental_Vector_Fields.pdf
Loading PDF 38/60: [24] Simultaneous_Position_and_Orientation_Planning_of_Nonholonomic_Multirobot_Systems_A_Dynamic_Vector_Field_Approach.pdf
Loading PDF 39/60: [37] Distributed_Nonlinear_Trajectory_Optimization_for_Multi-Robot_Motion_Planning.pdf
Loading PDF 40/60: [12] A_Distributed_Multi-Robot_Framework_for_Exploration_Information_Acquisition_and_Consensus.pdf
Loading PDF 41/60: [28] Highly_Efficient_Observation_Process_Based_on_FFT_Filtering_for_Robot_Swarm_Collaborative_Navigation_in_Unknown_Environments.pdf
Loading PDF 42/60: [5] Structured_

Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 141 0 (offset 0)
Ignoring wrong pointing object 249 0 (offset 0)
Ignoring wrong pointing object 280 0 (offset 0)
Ignoring wrong pointing object 293 0 (offset 0)
Ignoring wrong pointing object 300 0 (offset 0)
Ignoring wrong pointing object 391 0 (offset 0)
Ignoring wrong pointing object 416 0 (offset 0)
Ignoring wrong pointing object 531 0 (offset 0)


Loading PDF 47/60: [49] NeurIPS-2019-necessary-and-sufficient-geometries-for-gradient-methods-Paper.pdf
Loading PDF 48/60: [54] Multirobot_Symmetric_Formations_for_Gradient_and_Hessian_Estimation_With_Application_to_Source_Seeking.pdf
Loading PDF 49/60: [48] Importance Sampling1608.08814v1.pdf
Loading PDF 50/60: [60] swarm intelligence a review.pdf
Loading PDF 51/60: [32] khatib-1986-real-time-obstacle-avoidance-for-manipulators-and-mobile-robots.pdf
Loading PDF 52/60: [15] Distributing_Collaborative_Multi-Robot_Planning_With_Gaussian_Belief_Propagation.pdf
Loading PDF 53/60: [43] partial eigenstructure math-06-10-647.pdf
Loading PDF 54/60: [9] Multirobot_Field_of_View_Control_With_Adaptive_Decentralization.pdf
Loading PDF 55/60: [38] Distributed_Competition_of_Multi-Robot_Coordination_Under_Variable_and_Switching_Topologies.pdf
Loading PDF 56/60: [18] A survey of distributed optimization methods.pdf
Loading PDF 57/60: [50] 2402.11858v5.pdf
Loading PDF 58/60: [10] Singularity-Free_Guid

In [14]:
# Literature Meta-Analysis and Research Positioning
import re
import json
from collections import Counter, defaultdict
import pandas as pd
from datetime import datetime

def load_paper_summaries(file_path="all_paper_summaries.md"):
    """Load and parse the paper summaries from markdown file"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Split by paper sections
        papers = []
        sections = content.split('# Analysis for ')
        
        for section in sections[1:]:  # Skip the header section
            lines = section.split('\n')
            if lines:
                filename = lines[0].strip()
                paper_content = '\n'.join(lines[1:])
                papers.append({
                    'filename': filename,
                    'content': paper_content
                })
        
        print(f"Loaded {len(papers)} paper summaries")
        return papers
        
    except FileNotFoundError:
        print(f"Error: Could not find {file_path}")
        return []
    except Exception as e:
        print(f"Error loading summaries: {e}")
        return []

def extract_publication_years(papers):
    """Extract publication years from filenames and content"""
    years = []
    year_pattern = r'(19|20)\d{2}'
    
    for paper in papers:
        # Try to find year in filename first
        filename_years = re.findall(year_pattern, paper['filename'])
        content_years = re.findall(year_pattern, paper['content'][:500])  # Check first 500 chars
        
        found_years = filename_years + content_years
        if found_years:
            # Take the most recent reasonable year
            valid_years = [int(y) for y in found_years if 1990 <= int(y) <= 2025]
            if valid_years:
                years.append(max(valid_years))
            else:
                years.append(None)
        else:
            years.append(None)
    
    return years

def analyze_field_evolution(papers):
    """Analyze how the field has evolved over time"""
    years = extract_publication_years(papers)
    year_counts = Counter([y for y in years if y is not None])
    
    print("=== FIELD EVOLUTION ANALYSIS ===")
    print(f"Papers with identifiable years: {len([y for y in years if y is not None])}/{len(papers)}")
    
    if year_counts:
        print("\nPublication Timeline:")
        for year in sorted(year_counts.keys()):
            print(f"  {year}: {year_counts[year]} papers")
        
        # Identify trends by decade
        decades = defaultdict(int)
        for year in year_counts:
            decade = (year // 10) * 10
            decades[decade] += year_counts[year]
        
        print("\nDecade Distribution:")
        for decade in sorted(decades.keys()):
            print(f"  {decade}s: {decades[decade]} papers")
    
    return year_counts

def extract_themes_and_methods(papers):
    """Extract common themes and methodological approaches"""
    
    # Keywords to look for in different categories
    theme_keywords = {
        'Multi-Robot Systems': ['multi-robot', 'multi robot', 'swarm', 'collective', 'distributed', 'collaborative'],
        'Navigation & Control': ['navigation', 'path planning', 'control', 'guidance', 'steering'],
        'Vector Fields': ['vector field', 'flow field', 'gradient', 'potential field'],
        'Environmental Sensing': ['environmental', 'sensing', 'monitoring', 'tracking'],
        'Adaptive Systems': ['adaptive', 'learning', 'reinforcement', 'online'],
        'Localization': ['localization', 'SLAM', 'positioning', 'mapping'],
        'Formation Control': ['formation', 'consensus', 'coordination', 'synchronization'],
        'Source Seeking': ['source seeking', 'source finding', 'plume tracking', 'gradient following']
    }
    
    method_keywords = {
        'Machine Learning': ['neural network', 'deep learning', 'reinforcement learning', 'ML', 'AI'],
        'Kalman Filtering': ['kalman', 'EKF', 'UKF', 'particle filter'],
        'Optimization': ['optimization', 'optimal', 'minimize', 'maximize', 'genetic algorithm'],
        'Game Theory': ['game theory', 'nash equilibrium', 'cooperative', 'non-cooperative'],
        'Graph Theory': ['graph', 'network', 'connectivity', 'topology'],
        'Lyapunov Methods': ['lyapunov', 'stability', 'convergence'],
        'Simulation': ['simulation', 'gazebo', 'matlab', 'simulink'],
        'Real-world Testing': ['experiment', 'testbed', 'hardware', 'real robot', 'physical']
    }
    
    theme_counts = defaultdict(int)
    method_counts = defaultdict(int)
    
    for paper in papers:
        content_lower = paper['content'].lower()
        
        # Count themes
        for theme, keywords in theme_keywords.items():
            for keyword in keywords:
                if keyword in content_lower:
                    theme_counts[theme] += 1
                    break  # Count each theme only once per paper
        
        # Count methods
        for method, keywords in method_keywords.items():
            for keyword in keywords:
                if keyword in content_lower:
                    method_counts[method] += 1
                    break  # Count each method only once per paper
    
    return dict(theme_counts), dict(method_counts)

def extract_research_gaps(papers):
    """Extract commonly mentioned research gaps and limitations"""
    gap_indicators = [
        'limitation', 'challenge', 'future work', 'gap', 'problem', 
        'difficulty', 'issue', 'constraint', 'bottleneck'
    ]
    
    common_gaps = []
    
    for paper in papers:
        content_lower = paper['content'].lower()
        
        # Look for sections about research gaps
        gap_sections = []
        lines = content_lower.split('\n')
        
        for i, line in enumerate(lines):
            if any(indicator in line for indicator in gap_indicators):
                # Extract context around the gap mention
                start = max(0, i-1)
                end = min(len(lines), i+3)
                context = ' '.join(lines[start:end])
                gap_sections.append(context)
        
        common_gaps.extend(gap_sections)
    
    return common_gaps

def create_research_taxonomy(theme_counts, method_counts, total_papers):
    """Create a taxonomy of research streams"""
    
    print("\n=== RESEARCH TAXONOMY ===")
    
    print("\n1. PRIMARY RESEARCH THEMES:")
    sorted_themes = sorted(theme_counts.items(), key=lambda x: x[1], reverse=True)
    for theme, count in sorted_themes:
        percentage = (count / total_papers) * 100
        print(f"   • {theme}: {count} papers ({percentage:.1f}%)")
    
    print("\n2. METHODOLOGICAL APPROACHES:")
    sorted_methods = sorted(method_counts.items(), key=lambda x: x[1], reverse=True)
    for method, count in sorted_methods:
        percentage = (count / total_papers) * 100
        print(f"   • {method}: {count} papers ({percentage:.1f}%)")
    
    return sorted_themes, sorted_methods

def analyze_my_research_positioning(papers, my_field_description):
    """Analyze how my research positions relative to existing work"""
    
    print("\n" + "="*80)
    print("MY RESEARCH POSITIONING ANALYSIS")
    print("="*80)
    
    # Prepare analysis prompt
    positioning_analysis = f"""
Based on the literature analysis, I will now analyze how your research positions:

YOUR RESEARCH: {my_field_description}

LITERATURE THEMES FOUND:
"""
    
    # Add theme analysis
    theme_counts, method_counts = extract_themes_and_methods(papers)
    
    for theme, count in sorted(theme_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
        positioning_analysis += f"- {theme}: {count} papers\n"
    
    # Create the positioning prompt
    positioning_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert academic analyst specializing in research positioning and novelty assessment."),
        ("human", f"""
Based on this literature analysis of {len(papers)} papers, analyze my research positioning:

MY RESEARCH FIELD: {my_field_description}

LITERATURE ANALYSIS SUMMARY:
{positioning_analysis}

Please provide a detailed analysis addressing:

1. **UNIQUE POSITIONING**: How does my work differ from existing approaches? Be specific about what makes it unique.

2. **GAP FILLING**: Which specific gaps from the literature does my work address? Quote specific limitations you can identify.

3. **METHODOLOGICAL ADVANCES**: What novel aspects of my approach aren't covered in existing work?

4. **COMPARATIVE ADVANTAGES**: Compare my method to the 3-5 most similar approaches from the literature.

5. **POTENTIAL CRITICISMS**: What might reviewers say about novelty based on this literature? What are the strongest potential objections?

6. **RESEARCH POSITIONING STRATEGY**: How should I position this work in papers to maximize perceived novelty and impact?

Be thorough and critical. I want honest assessment of both strengths and potential weaknesses.
        """)
    ])
    
    return positioning_prompt

def run_comprehensive_analysis(summaries_file="all_paper_summaries.md"):
    """Run the complete meta-analysis"""
    
    print("COMPREHENSIVE LITERATURE META-ANALYSIS")
    print("="*60)
    
    # Load papers
    papers = load_paper_summaries(summaries_file)
    if not papers:
        return
    
    # 1. Field Evolution Analysis
    year_counts = analyze_field_evolution(papers)
    
    # 2. Theme and Method Analysis
    theme_counts, method_counts = extract_themes_and_methods(papers)
    
    # 3. Create Research Taxonomy
    sorted_themes, sorted_methods = create_research_taxonomy(theme_counts, method_counts, len(papers))
    
    # 4. Research Gaps Analysis
    print("\n=== COMMONLY MENTIONED RESEARCH GAPS ===")
    gaps = extract_research_gaps(papers)
    
    # Count most common gap-related terms
    gap_words = []
    for gap in gaps:
        gap_words.extend(gap.split())
    
    gap_word_counts = Counter([word for word in gap_words if len(word) > 4])
    print("\nMost frequently mentioned challenges/gaps:")
    for word, count in gap_word_counts.most_common(15):
        if count > 2:  # Only show words mentioned multiple times
            print(f"   • '{word}': {count} mentions")
    
    # 5. Create summary report
    create_summary_report(papers, theme_counts, method_counts, year_counts, gaps)
    
    return papers, theme_counts, method_counts, year_counts

def create_summary_report(papers, theme_counts, method_counts, year_counts, gaps):
    """Create a comprehensive summary report"""
    
    report = f"""
# LITERATURE META-ANALYSIS SUMMARY REPORT
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## OVERVIEW
- Total papers analyzed: {len(papers)}
- Papers with publication years: {len([y for y in extract_publication_years(papers) if y is not None])}

## TOP RESEARCH THEMES
"""
    
    for theme, count in sorted(theme_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
        percentage = (count / len(papers)) * 100
        report += f"- **{theme}**: {count} papers ({percentage:.1f}%)\n"
    
    report += "\n## TOP METHODOLOGICAL APPROACHES\n"
    
    for method, count in sorted(method_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
        percentage = (count / len(papers)) * 100
        report += f"- **{method}**: {count} papers ({percentage:.1f}%)\n"
    
    if year_counts:
        report += "\n## TEMPORAL DISTRIBUTION\n"
        for year in sorted(year_counts.keys(), reverse=True)[:10]:
            report += f"- **{year}**: {year_counts[year]} papers\n"
    
    # Save report
    with open("literature_meta_analysis_report.md", "w", encoding='utf-8') as f:
        f.write(report)
    
    print(f"\n✓ Comprehensive report saved to: literature_meta_analysis_report.md")

def analyze_research_positioning_with_llm(papers):
    """Use LLM to analyze research positioning"""
    
    # Extract key information for positioning analysis
    theme_counts, method_counts = extract_themes_and_methods(papers)
    gaps = extract_research_gaps(papers)
    years = extract_publication_years(papers)
    
    # Create comprehensive context for LLM
    literature_context = f"""
LITERATURE ANALYSIS SUMMARY ({len(papers)} papers):

TOP RESEARCH THEMES:
"""
    
    for theme, count in sorted(theme_counts.items(), key=lambda x: x[1], reverse=True)[:8]:
        percentage = (count / len(papers)) * 100
        literature_context += f"- {theme}: {count} papers ({percentage:.1f}%)\n"
    
    literature_context += "\nTOP METHODS:\n"
    for method, count in sorted(method_counts.items(), key=lambda x: x[1], reverse=True)[:8]:
        percentage = (count / len(papers)) * 100
        literature_context += f"- {method}: {count} papers ({percentage:.1f}%)\n"
    
    # Add time span if years are available
    valid_years = [y for y in years if y]
    if valid_years:
        literature_context += f"\nTIME SPAN: {min(valid_years)}-{max(valid_years)} (identifiable papers)"
    else:
        literature_context += f"\nTIME SPAN: Publication years not identifiable from filenames"
    
    # Create positioning analysis prompt
    positioning_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert academic analyst specializing in research positioning, novelty assessment, and competitive analysis for academic publications."),
        ("human", f"""
Based on this comprehensive literature analysis, provide a detailed research positioning analysis:

MY RESEARCH: {MY_FIELD_DESCRIPTION}

{literature_context}

SAMPLE RESEARCH GAPS IDENTIFIED:
{chr(10).join(gaps[:5])}

Provide detailed analysis for:

1. **UNIQUE POSITIONING**: How does my multirobot adaptive navigation approach differ from existing work? What makes the "current sensor data only, no memory" approach novel?

2. **GAP FILLING**: Which specific literature gaps does my work address? Be explicit about limitations in existing work.

3. **METHODOLOGICAL ADVANCES**: What's novel about using only current sensor readings for adaptive navigation in vector fields?

4. **COMPETITIVE COMPARISON**: Create a detailed comparison table showing my approach vs. the 3-5 most similar methods from this literature.

5. **REVIEWER CONCERNS**: What might reviewers criticize about novelty? What are the strongest potential objections?

6. **POSITIONING STRATEGY**: How should I frame this work to maximize perceived contribution and impact?

Be thorough, critical, and specific. Reference the literature analysis data provided.
        """)
    ])
    
    # Use the same LLM from the main script
    try:
        chain = positioning_prompt | llm
        response = chain.invoke({})
        positioning_analysis = response.content if hasattr(response, 'content') else str(response)
        
        # Save positioning analysis
        with open("research_positioning_analysis.md", "w", encoding='utf-8') as f:
            f.write(f"# RESEARCH POSITIONING ANALYSIS\n\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            f.write(positioning_analysis)
        
        print("\n" + "="*80)
        print("RESEARCH POSITIONING ANALYSIS")
        print("="*80)
        print(positioning_analysis)
        print(f"\n✓ Analysis saved to: research_positioning_analysis.md")
        
        return positioning_analysis
        
    except Exception as e:
        print(f"Error in LLM positioning analysis: {e}")
        return None

# Main execution
if __name__ == "__main__":
    print("Starting comprehensive literature meta-analysis...")
    
    # Run the analysis
    papers, theme_counts, method_counts, year_counts = run_comprehensive_analysis()
    
    # Generate LLM-based positioning analysis
    if papers:
        print("\nGenerating AI-powered research positioning analysis...")
        positioning_analysis = analyze_research_positioning_with_llm(papers)
    
    print("\n" + "="*60)
    print("ANALYSIS COMPLETE!")
    print("="*60)
    print("Generated files:")
    print("- literature_meta_analysis_report.md")
    print("- research_positioning_analysis.md")
    print("\nReview these files for comprehensive insights into your research positioning!")

Starting comprehensive literature meta-analysis...
COMPREHENSIVE LITERATURE META-ANALYSIS
Loaded 30 paper summaries
=== FIELD EVOLUTION ANALYSIS ===
Papers with identifiable years: 0/30

=== RESEARCH TAXONOMY ===

1. PRIMARY RESEARCH THEMES:
   • Navigation & Control: 30 papers (100.0%)
   • Vector Fields: 30 papers (100.0%)
   • Environmental Sensing: 30 papers (100.0%)
   • Adaptive Systems: 30 papers (100.0%)
   • Localization: 30 papers (100.0%)
   • Multi-Robot Systems: 29 papers (96.7%)
   • Formation Control: 28 papers (93.3%)
   • Source Seeking: 4 papers (13.3%)

2. METHODOLOGICAL APPROACHES:
   • Real-world Testing: 30 papers (100.0%)
   • Simulation: 28 papers (93.3%)
   • Graph Theory: 16 papers (53.3%)
   • Lyapunov Methods: 15 papers (50.0%)
   • Optimization: 14 papers (46.7%)
   • Game Theory: 6 papers (20.0%)
   • Kalman Filtering: 3 papers (10.0%)
   • Machine Learning: 3 papers (10.0%)

=== COMMONLY MENTIONED RESEARCH GAPS ===

Most frequently mentioned challenges/ga